In [15]:
import pandas as pd

In [16]:
data = pd.read_csv("USDKZT_200401_200501.csv", delimiter=',')
data.describe()

,<DATE>,<TIME>,<OPEN>,<HIGH>,<LOW>,<CLOSE>,<VOL>
count,1.626300e+04,16263.000000,16263.000000,16263.000000,16263.000000,16263.000000,16263.000000
mean,2.019322e+07,116017.173953,386.006057,386.042580,385.962268,386.001839,1.962983
std,4.121290e+03,53038.051626,12.890804,12.908755,12.878105,12.897320,3.892130
min,2.019050e+07,0.000000,374.870000,374.870000,374.870000,374.870000,0.000000
25%,2.019063e+07,80600.000000,379.600000,379.610000,379.560000,379.600000,1.000000
50%,2.019092e+07,111900.000000,383.760000,383.760000,383.460000,383.460000,1.000000
75%,2.020011e+07,145000.000000,387.060000,387.060000,387.060000,387.060000,1.000000
max,2.020050e+07,235900.000000,464.520000,465.020000,463.520000,465.020000,41.000000
